# 第8章 集成方法

在这一章中，我们将探讨一下主题：
- 理解集成——挂袋法
- 理解集成——提升法
- 理解集成——梯度提升

## 8.1 简介

本章主要内容将覆盖集成方法。  

基本的思路是拥有大量的模型，每一个都在训练集上产生差别不大的结果，一些模型相较其他的在某些方面的数据效果会更好一些。可以相信，最后从多个模型得到的输出结果肯定比仅从一个模型获得的结果要好。  

## 8.2 理解集成——挂袋法

假定数据自举 $m$ 次，这样就有 $m$ 个模型，也就有 $m$ 个 $y$，最后的预测值计算公式如下：
$$Y_{final(x)} = \frac{1}{m}\sum_{i=1}^my_m(x)$$

随机化是用来在建模过程中引入变化的另一种技术，一个例子就是在集成的每个模型里随机选择属性的子集，这样，不同模型使用不同的属性集合。这种技术被称为随机子空间方法。

#### 8.2.1 准备工作

#### 8.2.2 操作方法

In [2]:
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report
from sklearn.cross_validation import train_test_split

In [3]:
def get_data():
    """
    Make a sample classification dataset
    Returns : Independent variable y, dependent variable x
    """
    no_features = 30
    redundant_features = int(0.1 * no_features)
    informative_features = int(0.6 * no_features)
    repeated_features = int(0.1 * no_features)
    print(no_features, redundant_features, informative_features, repeated_features)
    x, y = make_classification(n_samples=500,n_features=no_features,flip_y=0.03,n_informative=informative_features,\
                               n_redundant=redundant_features,n_repeated=repeated_features,random_state=7)
    return x,y    

In [4]:
def build_single_model(x,y):
    model = KNeighborsClassifier()
    model.fit(x,y)
    return model

In [5]:
def build_bagging_model(x,y):
    bagging = BaggingClassifier(KNeighborsClassifier(),n_estimators=100,random_state=9,max_samples=1.0,max_features=0.7,bootstrap=True,\
                               bootstrap_features=True)
    bagging.fit(x,y)
    return bagging

In [6]:
def view_model(model):
    print("\n Sampled attributes in top 10 estimators\n")
    for i,feature_set in enumerate(model.estimators_features_[0:10]):
        print("estimator %d"%(i+1), feature_set)

In [8]:
x, y = get_data()

# 将数据划分为训练集、dev集和测试集
x_train,x_test_all,y_train,y_test_all = train_test_split(x,y,test_size=0.3,random_state=9)
x_dev,x_test,y_dev,y_test = train_test_split(x_test_all,y_test_all,test_size=0.3,random_state=9)

# 构建单个模型
model = build_single_model(x_train,y_train)
predicted_y = model.predict(x_train)
print("\n Single Model Accuracy on training data\n")
print(classification_report(y_train,predicted_y))

# 构建多个模型
bagging = build_bagging_model(x_train,y_train)
predicted_y = bagging.predict(x_train)
print("\n Bagging Model Accuracy on training data\n")
print(classification_report(y_train,predicted_y))
view_model(bagging)

# 查看dev集上运行的情况
print("\n Single Model Accuracy on Dev data\n")
predicted_y = model.predict(x_dev)
print(classification_report(y_dev,predicted_y))

print("\n Bagging Model Accuracy on Dev data\n")
predicted_y = bagging.predict(x_dev)
print(classification_report(y_dev,predicted_y))

30 3 18 3

 Single Model Accuracy on training data

             precision    recall  f1-score   support

          0       0.88      0.87      0.88       181
          1       0.87      0.88      0.87       169

avg / total       0.87      0.87      0.87       350


 Bagging Model Accuracy on training data

             precision    recall  f1-score   support

          0       0.93      0.97      0.95       181
          1       0.96      0.92      0.94       169

avg / total       0.95      0.95      0.95       350


 Sampled attributes in top 10 estimators

estimator 1 [25 20 10  6 17 18 11 17  9 14  3 10 10 23 22 18 17 11 21 20  1]
estimator 2 [14  3 27 28 20 20 27 25  0 21  1 12 20 21 29  1  0 28 16  4  9]
estimator 3 [29  5 23 19  2 16 21  4 13 27  1 15 24  5 14  1  4 25 22 26 29]
estimator 4 [23 10 16  7 22 11  0 14 14 17  8 17 27 12 13 23  8  7 27  0 27]
estimator 5 [ 3  0 26 13 23  7 27 15 18 11 26 18 26  3 22  6 11 21  6 12 19]
estimator 6 [16  5 24 19 21  2  2 22 12 21 14 2

#### 8.2.3 工作原理

In [ ]:
no_features = 30
redundant_features = int(0.1 * no_features)
informative_features = int(0.6 * no_features)
repeated_features = int(0.1 * no_features)

# n_samples是所需的实例数量
# n_features是每个实例需要的属性数量
# flip_y要求随机互换实例的%3，这是为了在数据中产生一些噪音。
# n_informative指定了从30个特征中选择具有足够的信息量来进行分类的特征个数
# n_redundant是关于冗余参数的，它们产生了高信息量特征的线性组合以构成特征之间的关联。
# n_repeated重复特征是从高信息量特征和冗余特征中随机选择的副本。
x, y = make_classification(n_samples=500,n_features=no_features,flip_y=0.03,n_informative=informative_features,\
                               n_redundant=redundant_features,n_repeated=repeated_features,random_state=7)

#### 8.2.4 更多内容

#### 8.2.5 参考资料